In [2]:
import pandas as pd
# import mysql.connector as mycon
import kagglehub
from kagglehub import KaggleDatasetAdapter

# This pulls in the wildfire data and reads it

In [3]:
use_cols = ['OBJECTID', 'FIRE_NAME', 'LATITUDE', 'LONGITUDE', 'DISCOVERY_DATE', 'NWCG_CAUSE_CLASSIFICATION', 'NWCG_GENERAL_CAUSE', 'FIRE_SIZE', 
            'FIRE_SIZE_CLASS', 'OWNER_DESCR', 'STATE', 'FIPS_NAME']

# Download latest version
wildfire_df = kagglehub.load_dataset(handle = "behroozsohrabi/us-wildfire-records-6th-edition", path = "data.csv", 
                                   adapter = KaggleDatasetAdapter.PANDAS, 
                                   pandas_kwargs={"usecols": use_cols, "compression": "zip"})

c:\Users\Taylor\AppData\Local\Programs\Python\Python313\Lib\site-packages\kagglehub\pandas_datasets.py:91: DtypeWarning: Columns (14,38) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


In [4]:
# wildfire_df = pd.read_csv('data.csv')
# Convert DISCOVERY_DATE to datetime object to remove records prior to year 2000

wildfire_df['DISCOVERY_DATE'] = pd.to_datetime(wildfire_df['DISCOVERY_DATE'], format = ('%m/%d/%Y'))

wildfire_df = wildfire_df[wildfire_df['DISCOVERY_DATE'].dt.year > 2000]

In [5]:
# Rename some columns for clarity

wildfire_df = wildfire_df.rename(columns = {"NWCG_CAUSE_CLASSIFICATION": "CAUSE_CLASSIFICATION", "NWCG_GENERAL_CAUSE": "SPECIFIC_CAUSE", 
                                            "OWNER_DESCR": "RESPONSIBLE_ENTITY", "FIPS_NAME": "COUNTY"})

# This pulls in housing data

In [6]:
bottom_tier_housing = pd.read_parquet('data/bottom_housing.parquet')
top_tier_housing = pd.read_parquet('data/top_housing.parquet')

## Remove Columns we don't use

In [7]:
remove_cols = ['RegionType', 'StateName']

bottom_tier_housing.drop(columns = remove_cols, inplace = True)
top_tier_housing.drop(columns = remove_cols, inplace = True)

## Remove rows with states we aren't using

In [8]:
states = ['CA', 'TX', 'GA', 'FL', 'AZ']

filtered_bottom_tier = bottom_tier_housing[bottom_tier_housing['State'].isin(states)]
filtered_top_tier = top_tier_housing[top_tier_housing['State'].isin(states)]


# Rent Data (load and remove columns'state values)

In [9]:
rent_index = pd.read_csv("data/Observed Rent Index by City.csv")

In [10]:
# Columns and state values to remove are same as housing sales data

rent_index.drop(columns = remove_cols, inplace = True)
filtered_rent_index = rent_index[rent_index['State'].isin(states)]

# Load Population Data into dictionary

In [52]:
state_population_dict = {}

for state in states:
    state_population_dict[state] = pd.read_csv(f"data/{state} City population estimates.csv")

In [53]:
columns_to_drop = ['SUMLEV', 'COUSUB', 'CONCIT', 'PRIMGEO_FLAG', 'FUNCSTAT', 'STNAME', 'ESTIMATESBASE2010', 'POPESTIMATE2010', 'STATE', 'COUNTY', 'PLACE']

## Iterate through df's and create PLACES_ID column and drop unnecessary columns

In [54]:
for key, value in state_population_dict.items():
    df = state_population_dict[f"{key}"]
    df['PLACES_ID'] = df[['STATE', 'COUNTY', 'PLACE']].astype('str').agg('-'.join, axis = 1)
    df.drop(columns = columns_to_drop, inplace = True)
    df.insert(0, 'PLACES_ID', df.pop('PLACES_ID'))